In [1]:
from sampo.generator.base import SimpleSynthetic
from sampo.generator.environment.contractor_by_wg import get_contractor_by_wg
from sampo.generator.types import SyntheticGraphType
from sampo.scheduler.heft.base import HEFTScheduler
from sampo.pipeline import SchedulingPipeline

# 1. Data preparation

In [2]:
# SimpleSynthetic object used for the simple work graph structure generation

r_seed = 231
ss = SimpleSynthetic(r_seed)

# simple graph
# should generate general (average) type of graph with 10 clusters from 100 to 200 vertices each

simple_wg = ss.work_graph(mode=SyntheticGraphType.GENERAL,
                          cluster_counts=10,
                          bottom_border=100,
                          top_border=200)

contractors = [get_contractor_by_wg(simple_wg)]

scheduler = HEFTScheduler()

# 2. Local optimization
There are two types of local optimization in SAMPO: order and schedule.

### Scheduling order optimization
This local optimization should rearrange scheduling order to improve scheduling results.

In [3]:
from sampo.scheduler.utils.local_optimization import SwapOrderLocalOptimizer

local_optimizer = SwapOrderLocalOptimizer()

schedule = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .optimize_local(local_optimizer, range(0, 10)) \
    .schedule(scheduler) \
    .finish()

schedule.execution_time

Swapped 1 times!
Swapped 1 times!


8661

### Schedule optimization
This local optimization should recalculate parts of schedule to make it better.

In [4]:
from sampo.scheduler.timeline.just_in_time_timeline import JustInTimeTimeline
from sampo.scheduler.utils.local_optimization import ParallelizeScheduleLocalOptimizer

local_optimizer = ParallelizeScheduleLocalOptimizer(JustInTimeTimeline)

schedule = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .schedule(scheduler) \
    .optimize_local(local_optimizer, range(0, 5)) \
    .finish()

schedule.execution_time

Found! finish of project finish of project


8661

### Both
Using pipeline you can apply both type of optimizations.
You also can stack local optimizers, they should be applied sequentially.

In [5]:
from sampo.pipeline import SchedulingPipeline

order_optimizer = SwapOrderLocalOptimizer()
schedule_optimizer = ParallelizeScheduleLocalOptimizer(JustInTimeTimeline)

schedule = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .optimize_local(order_optimizer, range(0, simple_wg.vertex_count // 2)) \
    .optimize_local(order_optimizer, range(simple_wg.vertex_count // 2, simple_wg.vertex_count)) \
    .schedule(scheduler) \
    .optimize_local(schedule_optimizer, range(0, simple_wg.vertex_count // 2)) \
    .optimize_local(schedule_optimizer, range(simple_wg.vertex_count // 2, simple_wg.vertex_count)) \
    .finish()

schedule.execution_time

Swapped 27 times!
Swapped 33 times!
Swapped 13 times!
Swapped 14 times!
Found! temporary road engineering preparation
Found! finish of project elem of pipe_network
Found! finish of project elem of pipe_network
Found! finish of project elem of pipe_network
Found! finish of project elem of pipe_network
Found! pipe drainage tank
Found! pipe KTP and NEP
Found! finish of project high power line
Found! pipe block local automation
Found! pipe metering installation
Found! pipe block dosage inhibitor
Found! pipe KTP and NEP
Found! pipe metering installation
Found! pipe start filters system
Found! pipe block water distribution
Found! pipe drainage tank
Found! pipe firewall tank
Found! finish of project elem of pipe_network
Found! finish of project elem of pipe_network
Found! finish of project elem of pipe_network
Found! finish of project mast
Found! finish of project mast
Found! finish of project power network
Found! finish of project pipe
Found! finish of project pipe
Found! finish of project p

8661